In [18]:
%run tire_transform.ipynb

In [14]:
import numpy as np
import argparse
import cv2
import glob
import math


def detect_tires(output, gray_scale=False):
    # Hough Circles detection requires min radius and 
    # dp (inverse ratio of the accumulator resolution to the image resolution) are 
    # video dependent (pipeline-size dependent ---> video height dependent). Therefore we use default params.
    default_param = 1.2
    default_height = 260
    minr = 40
    
    scale = output.shape[0]/default_height
    
    
    gray = (cv2.cvtColor(output, cv2.COLOR_RGB2GRAY) if not gray_scale else output)

    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, default_param/scale, 100, minRadius=int(minr*scale), maxRadius=400, param1 = 100, param2 = 80)

    if circles is not None:
        circles = circles[0]
    else:
        return None
    
    circles = circles[~np.all(circles == 0, axis=1)]

    circles = np.round(circles[:]).astype("int")
    if (circles.size==0): return None
    
    filtred_circles = [True for i in range(circles.shape[0])]
    
    #filter out circels with high IoU
    iou_threshold = 0.2
    for i in range(circles.shape[0]):
        for j in range(circles.shape[0]):
            area = inter_area(circles[i], circles[j])
            if i !=j and area!=0:
                
                area_1 = math.pi* circles[i][2]**2
                area_2 = math.pi* circles[j][2]**2
                
                if area/min(area_1, area_2) > iou_threshold:
                    
                    if circles[i][2] > circles[j][2]:                      
                        filtred_circles[j] = False 
                    else:
                        filtred_circles[i] = False
                        

    return circles[filtred_circles]


def draw_circles(img, circles_array):
    if circles_array is None:
        return img

    output = img.copy()
    for (x, y, r) in circles_array:
        cv2.circle(output, (x, y), r, (255, 0, 0), 4)
        cv2.rectangle(output, (x - r, y - r), (x + r, y + r), (0, 255, 255), 0)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (255, 128, 0), -1)
    return output
    





In [12]:
from math import asin, sqrt, pi
def inter_area(A, B):
    # A, B - (x,y,r) of circle
    d = sqrt((B[0] - A[0])**2 + (B[0] - A[0])**2)

    if (d < A[2] + B[2]):
        
        r1_2 = A[2]**2
        r2_2 = B[2]**2
        
        if (d < abs(B[2] - A[2])):
            return pi * min(r1_2, r2_2)
        
        x = (r1_2 - r2_2 + d**2) / (2 * d)
        z = x**2
        y = sqrt(r1_2 - z)

        return r1_2 * asin(y / A[2]) + r2_2 * asin(y / B[2]) - y * (x + sqrt(z + r2_2 - r2_2))
    else:
        return 0



In [13]:
import cv2
import numpy as np

def video_creation_hough(input_path, output_path, warp_flag=False):

    trans_path = 'transformed_stream.mp4'
    index = 0
    
    if warp_flag:
        video_stream_path(input_path, trans_path, 45, (1, 0.5))
        input_path = trans_path
    
    cap = cv2.VideoCapture(input_path)
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")
            
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    #TODO: estimate frame width and height and proper Transform from first frames of the scene.
    out = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))
    
    while(True):
        
        index += 1
        if ((index % 50) == 0):
            print(index)
        
        ret, frame = cap.read()
        if ret == True: 
            detected = draw_circles(frame, detect_tires(frame))
            cv2.imshow('k', detected)
            out.write(detected)
                    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break 

    cap.release()
    out.release()

    cv2.destroyAllWindows()

In [7]:
#video_creation_hough("video_folder/ProcessedODSample2.mp4", "video_folder/HoughDetectedSample.mp4")

50
100
150
200
250
300
350
400
450
500


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
